In [1]:
from gurobipy import *

In [2]:
m1 = Model('VRPTW')

Restricted license - for non-production use only - expires 2023-10-25


In [3]:
from operator import le
from unicodedata import name
from numpy import conjugate
from ortools.linear_solver import pywraplp
from ortools.linear_solver import pywraplp
from datetime import datetime
import random
from gurobipy import *
m1 = Model('vrptw')

# 20 customer -> 1.1 minute
# 30 customers -> 5 minute    1.69 minutes
# 40                          2.45 min
# 50 customer -> 160 minute


def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw


def getNodesandCustomer(nodeCount, variables, x, edges, discreteTimeWindows, vehicleReq, timeWindows):
    goodVariables = []
    visited = []
    goodEdges = []

    for j in range(variables):
        if(x[j].solution_value() > 0 and j < len(edges)):
            goodVariables.append(j)

    print("good var:", goodVariables)
    print()

    for i in range(0, len(goodVariables)):
        visited.append(0)
        goodEdges.append(edges['x[%i]' % goodVariables[i]])

    print("good edges :", goodEdges)
    print(timeWindows)
    for i in range(0, vehicleReq):
        nextFirstNode = 0
        flag = 0
        NodesInPath = []
        while(nextFirstNode != -1):
            for j in range(0, len(goodEdges)):
                if(visited[j] == 0 and flag == 0):
                    nextFirstNode = goodEdges[j][-1]
                    NodesInPath.append(goodEdges[j][0])
                    visited[j] = 1
                    flag = 1
                elif(nextFirstNode == -1):
                    NodesInPath.append(-1)
                    break
                elif(visited[j] == 0 and goodEdges[j][0] == nextFirstNode):
                    nextFirstNode = goodEdges[j][-1]
                    NodesInPath.append(goodEdges[j][0])
                    visited[j] = 1

        print("path ", i, " ", NodesInPath)
        for j in range(1, len(NodesInPath)):
            currNode = NodesInPath[j]
            if(currNode != -1):
                for k in range(0, len(nodeCount)):
                    if(currNode in nodeCount[k]):
                        time = discreteTimeWindows[k][nodeCount[k].index(
                            currNode)]
                        print("Customer :", (k+1), " Time :", time,
                              "     valid time window", timeWindows[k])
                        break

        print()

    return 0



customers = 10

# timeWindows = [[20, 30], [30, 40], [30, 40]]
timeWindows = []
for i in range(0,customers):
    st = random.randint(1,2)*10
    et = random.randint(1,3)*10 + st
    tw = [st,et]
    timeWindows.append(tw)

print("time Windows ", timeWindows)
print()

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i, 10))

print("discreteTimeWindows", discreteTimeWindows)
print()
# starting_time = '6:00am'

# timeMatrix = [[0, 30, 10], [30, 0, 10], [10, 10, 0]]

timeMatrix = []
for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
            timeMatrix[i][j] = random.randint(1,10)*10
            timeMatrix[j][i] = timeMatrix[i][j]

print("timeMatrix", timeMatrix)
print()

count = 0
nodeCount = []  # assigns a number to node using count and stores its value

for i in range(0, len(discreteTimeWindows)):
    c = []
    for j in range(0, len(discreteTimeWindows[i])):
        c.append(count+1)
        count += 1
    nodeCount.append(c)

print("Node Count ", nodeCount)
print()

edgeFlows = {}
edgeNumber = 0
x = {}
edges = {}
edgesList = []  # stores edge number going out of ith node
edgesList.append([])  # for s
# solver = pywraplp.Solver.CreateSolver('GLOP')

# GRAPH CONSTRUCTION

# out of s to starting of each customer
e = []
for i in range(0, len(nodeCount)):
#     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
    x[edgeNumber] = m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber )
    m1.update()
    edges['x[%i]' % edgeNumber] = (0, nodeCount[i][0])
    edgeFlows[edgeNumber] = (0, nodeCount[i][0])
    e.append(edgeNumber)
    edgeNumber += 1
edgesList[0] = e

for i in range(0, len(nodeCount)):
    for j in range(0, len(nodeCount[i])-1):
        e = []
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x[edgeNumber] = m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber )
        m1.update()
        edges['x[%i]' % edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        edgeFlows[edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
        # add edge number in edge list
        e.append(edgeNumber)
        edgesList.append(e)
        edgeNumber += 1
    edgesList.append([])
print("Edges List:", edgesList)

    
for i in range(len(discreteTimeWindows)):
    tu = nodeCount[i][-1]
    start_utime = discreteTimeWindows[i][0]
    end_utime = discreteTimeWindows[i][-1]

    # stores edges reachable from u
    for k in range(0, len(discreteTimeWindows)):
        sv = nodeCount[k][0]

        start_vtime = discreteTimeWindows[k][0]
        end_vtime = discreteTimeWindows[k][-1]


        if(i != k and start_utime + timeMatrix[i][k] > end_vtime):
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)

              # not reachable

        elif(i != k and start_vtime > end_utime + timeMatrix[i][k]):
            # edge bw (u,tu) -> (v,sv)
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x[edgeNumber] = m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber )
            m1.update()
            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            edgeFlows[edgeNumber] = (tu, sv)
            print(tu, "->", sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif(i != k and start_vtime == end_utime + timeMatrix[i][k]):
            print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
            e = []
#             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x[edgeNumber] = m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber )
            m1.update()

            edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
            print(tu, "->", sv)
            edgeFlows[edgeNumber] = (tu, sv)
            e.append(edgeNumber)
            edgesList[tu].extend(e)
            edgeNumber += 1


        elif (i != k and start_vtime < end_utime + timeMatrix[i][k]):
            print("ishit bajpai")
            firstReachableNode_index = 0  # first such that node >= sv
            for j in range(0, len(discreteTimeWindows[i])):
                if(timeMatrix[i][k] + discreteTimeWindows[i][j] >= start_vtime):
                    firstReachableNode_index = j
                    break
            # now starting from firstReachableNode make edges until feasible
            for l in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                e = []
                reachingTime = discreteTimeWindows[i][l]+timeMatrix[i][k]
                for r in range(0, len(discreteTimeWindows[k])):
                    if(discreteTimeWindows[k][r] >= reachingTime):
                        print(edgesList[nodeCount[i][l]])
                        print(nodeCount[i][l], "->", nodeCount[k][r])
                        # edge bw these two
#                         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                        x[edgeNumber] = m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber )
                        m1.update()

                        edges['x[%i]' % edgeNumber] =(nodeCount[i][l], nodeCount[k][r])  # u -> v reachable
                        edgeFlows[edgeNumber] = (nodeCount[i][l], nodeCount[k][r])

                        e.append(edgeNumber)
                        edgesList[nodeCount[i][l]].extend(e)
                        print("hdhdh", edgesList[nodeCount[i][l]])
                        edgeNumber += 1
                        break




print('edge list b4 s t', edgesList)
print()

# into t (except s)
e = []
for i in range(0, len(nodeCount)):
#     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
    x[edgeNumber] = m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber )
    m1.update()

    edges['x[%i]' % edgeNumber] = (nodeCount[i][-1], -1)
    edgeFlows[edgeNumber] = (nodeCount[i][-1], -1)
    # add edge number in edge list
    edgesList[nodeCount[i][-1]].append(edgeNumber)
    edgeNumber += 1
edgesList.append([])

print("Node Count after s and t:", nodeCount)
print()

# GRAPH Construction OVER

# if I add it to  0 edge i'll get flow for 1 at the same edge  , add to 1 customer 2 customer
offset = edgeNumber
print("edge flows:", edgeFlows)

flows = [0]*customers
for i in range(customers):
    flows[i] = edgeFlows.copy()
    keys = edgeFlows.keys()
    for j in keys:  # stores key values of edgeflows i.e 0 1 2
        # all x[] > offset are for flow variables
#         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x[edgeNumber] = m1.addVar( lb=0, ub=1 , vtype= GRB.INTEGER, name='x[%i]' % edgeNumber )
        m1.update()

        # remove key and replace it by edge
        t = flows[i].pop(j)
        flows[i][edgeNumber] = t
        edgeNumber += 1

print("flows", flows)

print()

with open('edgeList.txt', 'w') as f:
    f.write("Edge List \n")
    f.write(str(edgesList))
with open('edges.txt', 'w') as f:
    f.write("Edges \n")
    f.write(str(edges))

print('edge list', edgesList)
print()
print("edges", edges)
print()
print('edge number', edgeNumber)
print()

data = {}

data['constraint_coeffs'] = []
data['bounds'] = []  # inclusive

# # contraint for outgoing flow for a customer

# for i in range(0, len(nodeCount)):  # choose one list from nodeCount
#     contraint = [0] * (edgeNumber)
#     for j in range(0, len(nodeCount[i])):  # se
#         node = nodeCount[i][j]
#         for k in range(0, len(edgesList[node])):
#             e = edgesList[node][k]
#             u = edges['x[%i]' % e][0]
#             v = edges['x[%i]' % e][1]
#                 # print("u", u, "v", v)
#             if(v not in nodeCount[i]):
#                     # print('hello')
#                     # print("u", u, "v", v)
#                     contraint[e] = 1

#     data['constraint_coeffs'].append(contraint)
#     data['bounds'].append('1u')
#     data['constraint_coeffs'].append(contraint)
#     data['bounds'].append('1l')

# # print(data['constraint_coeffs'])
# # print(data['bounds'])

# # constraint for incoming for a customer
# incomingEdgeList = []
# for i in range(0, len(edgesList)):
#     incomingEdgeList.append([])  # adding lists in a list
# for i in edges:
#     v = edges[i][1]  # u -> v

#     edge_no = int(i[2:-1])
#     incomingEdgeList[v].append(edge_no)

# print("incoming edge list: ", incomingEdgeList)
# print()

# for i in range(0, len(nodeCount)):
#     contraint = [0] * (edgeNumber)  # indicates number of edges
#     for j in range(0, len(nodeCount[i])):
#         node = nodeCount[i][j]
#         for k in range(0, len(incomingEdgeList[node])):
#             e = incomingEdgeList[node][k]
#             u = edges['x[%i]' % e][0]
#             v = edges['x[%i]' % e][1]
#                 # print("u", u, "v", v)
#             if(u not in nodeCount[i]):
#                     # print('hello')
#                     # print("u", u, "v", v)
#                     contraint[e] = 1

#     data['constraint_coeffs'].append(contraint)
#     data['bounds'].append('1u')
#     data['constraint_coeffs'].append(contraint)
#     data['bounds'].append('1l')

# # print(data['constraint_coeffs'])
# # print(data['bounds'])

# # constraint for a node incoming = outgoing
# for i in range(0, len(nodeCount)):
#     for j in range(0, len(nodeCount[i])):
#         contraint = [0] * (edgeNumber)
#         node = nodeCount[i][j]
#         for k in range(0, len(incomingEdgeList[node])):
#             e = incomingEdgeList[node][k]
#             contraint[e] = 1
#         for k in range(0, len(edgesList[node])):
#             e = edgesList[node][k]
#             contraint[e] = -1
#         data['constraint_coeffs'].append(contraint)
#         data['bounds'].append('0u')

# # print(data['constraint_coeffs'])
# # print(data['bounds'])

# # constraints for incoming = 1
# print("offset", offset)
# # print("flows",flows)
# # l = []
# # for z in range(len(contraint)):
# #         l.append(z % 10)
# # print(l)
# #  0<= xij - fij    f<x => 0<=x-f
# for j in range(len(flows)):
#     diff = offset*j+offset
#     for k in flows[j].keys():
#         contraint = [0]*edgeNumber
#         contraint[k] = -1
#         contraint[k-diff] = 1
#         data['constraint_coeffs'].append(contraint)
#         data['bounds'].append('0l')

#         # print(contraint," >= 0")

# # # constraint for incoming for a particular customer = 1
# print("CONSTRAINT for incoming = 1 ")
# for c in range(customers):
#     addn = offset*(c+1)
#     for i in range(0, len(nodeCount)):
#         # indicates number of edges
#         contraint = [0] * (edgeNumber)
#         for j in range(0, len(nodeCount[i])):
#             node = nodeCount[i][j]
#             for k in range(0, len(incomingEdgeList[node])):
#                 e = incomingEdgeList[node][k]
#                 u = edges['x[%i]' % e][0]
#                 v = edges['x[%i]' % e][1]
#                 # print("u", u, "v", v)
#                 if(u not in nodeCount[i]):
#                     # print('hello')
#                     # print("u", u, "v", v)
#                     contraint[e+addn] = 1

#         data['constraint_coeffs'].append(contraint)
#         data['bounds'].append('1u')
#         data['constraint_coeffs'].append(contraint)
#         data['bounds'].append('1l')


#         # print(contraint," =  1")

# # print("data[constraints] \n", data['constraint_coeffs'])
# # print("bounds\n",data['bounds'])
# # c=[]
# # for i in range(len(data['constraint_coeffs'][0])):
# #     c.append(i%10)
# # print(c)
# # for i in range(len(data['constraint_coeffs'])):
# #     print(data['constraint_coeffs'][i],"=",data['bounds'][i])

# # # flow conservation for node
# print("Constraint for node conservation")
# # l = []
# # for z in range(len(contraint)):
# #         l.append(z % 10)
# # print(l)
# for c in range(customers):
#     addn = offset*c+offset
#     for i in range(0, len(nodeCount)):
#         for j in range(0, len(nodeCount[i])):
#             contraint = [0] * (edgeNumber)
#             node = nodeCount[i][j]
#             for k in range(0, len(incomingEdgeList[node])):
#                 e = incomingEdgeList[node][k]
#                 contraint[e+addn] = 1
#             for k in range(0, len(edgesList[node])):
#                 e = edgesList[node][k]
#                 contraint[e+addn] = -1
#             data['constraint_coeffs'].append(contraint)
#             data['bounds'].append('0u')





# # objective function minimize outgoing flow from 0 node
# objectiveCoefficients = [0] * (edgeNumber)
# for i in range(0, len(edgesList[0])):
#     objectiveCoefficients[edgesList[0][i]] = 1
# data['obj_coeffs'] = objectiveCoefficients

# # print(objectiveCoefficients)

# data['num_vars'] = edgeNumber
# data['num_constraints'] = len(data['constraint_coeffs'])

# for i in range(data['num_constraints']):
#     if(data['bounds'][i][-1] == 'u'):
#         constraint = solver.RowConstraint(
#             0, int(data['bounds'][i][:-1]), '')
#     else:
#         constraint = solver.RowConstraint(
#             int(data['bounds'][i][:-1]), solver.infinity() , '')
#     for j in range(data['num_vars']):
#         constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
# print('Number of constraints =', solver.NumConstraints())

# objective = solver.Objective()
# for j in range(data['num_vars']):
#     objective.SetCoefficient(x[j], data['obj_coeffs'][j])
# objective.SetMinimization()

# status = solver.Solve()

# if status == pywraplp.Solver.OPTIMAL:
#     print('Objective value =', solver.Objective().Value())
#     # for j in range(data['num_vars']):
#     #     print(x[j].name(), ' = ', x[j].solution_value())
#     print('Problem solved in %f milliseconds' % solver.wall_time())
#     print('Problem solved in %d iterations' % solver.iterations())
#     # getNodesandCustomer(nodeCount,data['num_vars'],x,edges,discreteTimeWindows,int(solver.Objective().Value()),timeWindows)

#     print()

# else:
#     print('The problem does not have an optimal solution.')


# if __name__ == '__main__':
# main()

time Windows  [[20, 50], [20, 40], [10, 30], [10, 30], [10, 20], [20, 50], [20, 50], [20, 30], [10, 20], [20, 40]]

discreteTimeWindows [[20, 30, 40, 50], [20, 30, 40], [10, 20, 30], [10, 20, 30], [10, 20], [20, 30, 40, 50], [20, 30, 40, 50], [20, 30], [10, 20], [20, 30, 40]]

timeMatrix [[0, 30, 60, 90, 30, 90, 60, 30, 60, 90], [30, 0, 80, 80, 20, 30, 20, 100, 10, 100], [60, 80, 0, 20, 30, 40, 60, 30, 60, 100], [90, 80, 20, 0, 80, 20, 40, 100, 30, 40], [30, 20, 30, 80, 0, 60, 100, 100, 50, 20], [90, 30, 40, 20, 60, 0, 50, 60, 50, 10], [60, 20, 60, 40, 100, 50, 0, 90, 30, 20], [30, 100, 30, 100, 100, 60, 90, 0, 30, 10], [60, 10, 60, 30, 50, 50, 30, 30, 0, 80], [90, 100, 100, 40, 20, 10, 20, 10, 80, 0]]

Node Count  [[1, 2, 3, 4], [5, 6, 7], [8, 9, 10], [11, 12, 13], [14, 15], [16, 17, 18, 19], [20, 21, 22, 23], [24, 25], [26, 27], [28, 29, 30]]

Edges List: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10], [11], [12], [], [13], [14], [], [15], [16], [], [17], [18], [], [19], [], [20], [21], [22],

In [4]:
edgeFlows 
# edgeNumber = 0
# x = {}
# edges = {}
# edgesList = []  # stores edge number going out of ith node


{0: (0, 1),
 1: (0, 5),
 2: (0, 8),
 3: (0, 11),
 4: (0, 14),
 5: (0, 16),
 6: (0, 20),
 7: (0, 24),
 8: (0, 26),
 9: (0, 28),
 10: (1, 2),
 11: (2, 3),
 12: (3, 4),
 13: (5, 6),
 14: (6, 7),
 15: (8, 9),
 16: (9, 10),
 17: (11, 12),
 18: (12, 13),
 19: (14, 15),
 20: (16, 17),
 21: (17, 18),
 22: (18, 19),
 23: (20, 21),
 24: (21, 22),
 25: (22, 23),
 26: (24, 25),
 27: (26, 27),
 28: (28, 29),
 29: (29, 30),
 30: (5, 4),
 31: (5, 19),
 32: (5, 22),
 33: (6, 23),
 34: (8, 13),
 35: (8, 19),
 36: (11, 10),
 37: (11, 17),
 38: (12, 18),
 39: (13, 19),
 40: (11, 23),
 41: (14, 3),
 42: (15, 4),
 43: (14, 6),
 44: (15, 7),
 45: (14, 29),
 46: (15, 30),
 47: (16, 29),
 48: (17, 30),
 49: (20, 7),
 50: (20, 30),
 51: (24, 4),
 52: (24, 29),
 53: (25, 30),
 54: (26, 5),
 55: (27, 6),
 56: (26, 22),
 57: (27, 23),
 58: (28, 17),
 59: (29, 18),
 60: (30, 19),
 61: (28, 22),
 62: (29, 23),
 63: (28, 25),
 64: (4, -1),
 65: (7, -1),
 66: (10, -1),
 67: (13, -1),
 68: (15, -1),
 69: (19, -1),
 70

In [5]:
x

{0: <gurobi.Var x[0]>,
 1: <gurobi.Var x[1]>,
 2: <gurobi.Var x[2]>,
 3: <gurobi.Var x[3]>,
 4: <gurobi.Var x[4]>,
 5: <gurobi.Var x[5]>,
 6: <gurobi.Var x[6]>,
 7: <gurobi.Var x[7]>,
 8: <gurobi.Var x[8]>,
 9: <gurobi.Var x[9]>,
 10: <gurobi.Var x[10]>,
 11: <gurobi.Var x[11]>,
 12: <gurobi.Var x[12]>,
 13: <gurobi.Var x[13]>,
 14: <gurobi.Var x[14]>,
 15: <gurobi.Var x[15]>,
 16: <gurobi.Var x[16]>,
 17: <gurobi.Var x[17]>,
 18: <gurobi.Var x[18]>,
 19: <gurobi.Var x[19]>,
 20: <gurobi.Var x[20]>,
 21: <gurobi.Var x[21]>,
 22: <gurobi.Var x[22]>,
 23: <gurobi.Var x[23]>,
 24: <gurobi.Var x[24]>,
 25: <gurobi.Var x[25]>,
 26: <gurobi.Var x[26]>,
 27: <gurobi.Var x[27]>,
 28: <gurobi.Var x[28]>,
 29: <gurobi.Var x[29]>,
 30: <gurobi.Var x[30]>,
 31: <gurobi.Var x[31]>,
 32: <gurobi.Var x[32]>,
 33: <gurobi.Var x[33]>,
 34: <gurobi.Var x[34]>,
 35: <gurobi.Var x[35]>,
 36: <gurobi.Var x[36]>,
 37: <gurobi.Var x[37]>,
 38: <gurobi.Var x[38]>,
 39: <gurobi.Var x[39]>,
 40: <gurobi.Var x[4

In [6]:
edgesList

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10],
 [11],
 [12],
 [64],
 [13, 30, 31, 32],
 [14, 33],
 [65],
 [15, 34, 35],
 [16],
 [66],
 [17, 36, 37, 40],
 [18, 38],
 [39, 67],
 [19, 41, 43, 45],
 [42, 44, 46, 68],
 [20, 47],
 [21, 48],
 [22],
 [69],
 [23, 49, 50],
 [24],
 [25],
 [70],
 [26, 51, 52],
 [53, 71],
 [27, 54, 56],
 [55, 57, 72],
 [28, 58, 61, 63],
 [29, 59, 62],
 [60, 73],
 []]

In [7]:
discreteTimeWindows

[[20, 30, 40, 50],
 [20, 30, 40],
 [10, 20, 30],
 [10, 20, 30],
 [10, 20],
 [20, 30, 40, 50],
 [20, 30, 40, 50],
 [20, 30],
 [10, 20],
 [20, 30, 40]]

In [8]:
edgeNumber

814

In [9]:
for i in range(0, len(nodeCount)):
    print(nodeCount[i])

[1, 2, 3, 4]
[5, 6, 7]
[8, 9, 10]
[11, 12, 13]
[14, 15]
[16, 17, 18, 19]
[20, 21, 22, 23]
[24, 25]
[26, 27]
[28, 29, 30]


In [14]:
for i in range(1, len(nodeCount)):
#     print(nodeCount[i])
    
    for j in range(0, len(nodeCount[i])):
        node = nodeCount[i][j]
        for k in range(0,len(edgesList[node])):
            print(edgesList[node])
            
        break
    break

[13, 30, 31, 32]
[13, 30, 31, 32]
[13, 30, 31, 32]
[13, 30, 31, 32]


In [13]:
for i in range(1, len(nodeCount)):  # choose one list from nodeCount
    contraint = [0] * (edgeNumber)
    for j in range(0, len(nodeCount[i])):  # se
        node = nodeCount[i][j]
        for k in range(0, len(edgesList[node])):
            e = edgesList[node][k]
            u = edges['x[%i]' % e][0]
            v = edges['x[%i]' % e][1]
            print(e)
            print(u)
            print(v)
        break
    break


13
5
6
30
5
4
31
5
19
32
5
22


In [15]:
edges

{'x[0]': (0, 1),
 'x[1]': (0, 5),
 'x[2]': (0, 8),
 'x[3]': (0, 11),
 'x[4]': (0, 14),
 'x[5]': (0, 16),
 'x[6]': (0, 20),
 'x[7]': (0, 24),
 'x[8]': (0, 26),
 'x[9]': (0, 28),
 'x[10]': (1, 2),
 'x[11]': (2, 3),
 'x[12]': (3, 4),
 'x[13]': (5, 6),
 'x[14]': (6, 7),
 'x[15]': (8, 9),
 'x[16]': (9, 10),
 'x[17]': (11, 12),
 'x[18]': (12, 13),
 'x[19]': (14, 15),
 'x[20]': (16, 17),
 'x[21]': (17, 18),
 'x[22]': (18, 19),
 'x[23]': (20, 21),
 'x[24]': (21, 22),
 'x[25]': (22, 23),
 'x[26]': (24, 25),
 'x[27]': (26, 27),
 'x[28]': (28, 29),
 'x[29]': (29, 30),
 'x[30]': (5, 4),
 'x[31]': (5, 19),
 'x[32]': (5, 22),
 'x[33]': (6, 23),
 'x[34]': (8, 13),
 'x[35]': (8, 19),
 'x[36]': (11, 10),
 'x[37]': (11, 17),
 'x[38]': (12, 18),
 'x[39]': (13, 19),
 'x[40]': (11, 23),
 'x[41]': (14, 3),
 'x[42]': (15, 4),
 'x[43]': (14, 6),
 'x[44]': (15, 7),
 'x[45]': (14, 29),
 'x[46]': (15, 30),
 'x[47]': (16, 29),
 'x[48]': (17, 30),
 'x[49]': (20, 7),
 'x[50]': (20, 30),
 'x[51]': (24, 4),
 'x[52]':

In [ ]:
edgesList